In [ ]:
%%capture
!pip install opendatasets
!pip install imutils
!pip install facenet_pytorch

In [ ]:
import opendatasets as od
import zipfile, urllib.request, shutil, requests
import numpy as np
import pandas as pd
import os
from pathlib import Path
import IPython
import cv2
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import matplotlib.image as image
from tqdm import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()
import glob
from PIL import Image
from imutils.video import FileVideoStream
import time
import torch
from facenet_pytorch import MTCNN
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
os.mkdir('/content/data')
os.mkdir('/content/train')

In [ ]:
#links of the data parts to be downlaoded
parts = ["",
         ""
    ]

In [ ]:
#Donwload each part and save it in the files
for ind , part in enumerate(parts):
    response = requests.get(part, stream=True)
    with open(f'{ind}.zip', "wb") as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)

In [ ]:
#Extract the files from the downloaded parts then delete each part after extracing to save storage 
for ind in range(len(parts)):
    f = f"/content/{ind}.zip"
    with zipfile.ZipFile(f, 'r') as zip_ref:
        zip_ref.extractall("/content/data")
    os.remove(f)

In [ ]:
class FastMTCNN(object):
    """Fast MTCNN implementation."""
    
    def __init__(self, stride, resize=1, *args, **kwargs):
        """Constructor for FastMTCNN class.
        
        Arguments:
            stride (int): The detection stride. Faces will be detected every `stride` frames
                and remembered for `stride-1` frames.
        
        Keyword arguments:
            resize (float): Fractional frame scaling. [default: {1}]
            *args: Arguments to pass to the MTCNN constructor. See help(MTCNN).
            **kwargs: Keyword arguments to pass to the MTCNN constructor. See help(MTCNN).
        """
        self.stride = stride
        self.resize = resize
        self.mtcnn = MTCNN(*args, **kwargs)
        
    def __call__(self, frames):
        """Detect faces in frames using strided MTCNN."""
        if self.resize != 1:
            frames = [
                cv2.resize(f, (int(f.shape[1] * self.resize), int(f.shape[0] * self.resize)))
                    for f in frames
            ]
                      
        boxes, probs = self.mtcnn.detect(frames[::self.stride])
        faces = []
        for i, frame in enumerate(frames):
            box_ind = int(i / self.stride)
            if boxes[box_ind] is None:
                continue
            for box in boxes[box_ind]:
                box = [int(b) for b in box]
                faces.append(frame[box[1]:box[3], box[0]:box[2]])
        return faces

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
fast_mtcnn = FastMTCNN(
stride=4,
resize=0.5,
margin=50,
factor=0.6,
keep_all=True,
device=device,
min_face_size = 60
)

In [ ]:
def save_vid(frame_array,filename):
    if len(frame_array) < 60:
        pass
    frames = 0
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    out = cv2.VideoWriter('/content/train/'+os.path.basename(filename),fourcc, 60, (240,240))
    for frame in frame_array:
        try:
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            frame = cv2.resize(frame,(240,240))
            out.write(frame)
            frames +=1
        except: 
            pass
        if frames == 60:
            out.release()
            break

In [ ]:
def run_detection(fast_mtcnn, filenames):
    frames = []
    frames_processed = 0
    faces_detected = 0
    batch_size = 60
    start = time.time()
    for filename in tqdm(filenames):
        v_cap = FileVideoStream(filename).start()
       # print(filename)
        #print(v_cap.read().shape)
        v_len = int(v_cap.stream.get(cv2.CAP_PROP_FRAME_COUNT))
        for j in range(min(60,v_len)):
            frame = v_cap.read()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
            if len(frames) >= batch_size or j == v_len - 1:
                faces = fast_mtcnn(frames)
                frames_processed += len(frames)
                faces_detected += len(faces)
                frames = []
                print(
                    f'Frames per second: {frames_processed / (time.time() - start):.3f},',
                    f'faces detected: {faces_detected}\r',
                    end=''
                )
                save_vid(faces,filename)
                
        v_cap.stop()

In [ ]:
f = glob.glob('/content/data/*/*')

In [ ]:
len(f)

10122

In [ ]:
run_detection(fast_mtcnn,f)

In [ ]:
shutil.make_archive('/content/cropped', 'zip', '/content/train/')